# <p style="padding:50px;background-color:#DA8359;margin:0;color:#fafefe;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:100">Setup</p>

In [56]:
# Imports
import re
import typing
from collections import Counter
from ghalatawi.autocorrector import AutoCorrector

In [ ]:
def extract_sentences_yt(file_path: str) -> list:
    '''
    process youtube documents

    @param file_path: a string represent file path
    '''

    with open(file_path, 'r', errors="ignore", encoding="utf-8") as f:
        text = f.read()

    # Remove numbers followed by ':'
    text = re.sub(r'\d+.*\d*\s*:', '', text)

    # Define sentence delimiters for Arabic
    sentence_endings = r'(?<=[.!؟؛،])\s+'

    # Split sentences while preserving dependencies
    sentences = re.split(sentence_endings, text)

    return [s.strip() for s in sentences if len(s.strip()) > 1] # Remove empty strings
    

In [ ]:
# Example
data = extract_sentences_yt("البطاطس  الدحيح.txt")
data

['لكل شخص في آخر الشهر،',
 'ما معاهوش غير 50 جنيه،',
 'ومحتاج ياكل أكلة تشبّعه...',
 'لكل واحد "فورمة"،',
 'محتاج أكلة سريعة الهضم،',
 'تدّيله طاقة،',
 'تخلّيه يكمّل التمرينة...',
 'لكل واحد مش عارف ياكل،',
 'ونفسه في أكلة جانبية مع الأكل،',
 'عشان تفتح نِفسه...',
 'العالم محتاج بطل خارق...',
 'بطل حقيقي يقدر ينقذهم.',
 'العالم محتاج...',
 'بطاطس.',
 'اللي ما معاهوش فلوس،',
 'ياكل سندوتش بطاطس سوري.',
 'اللي عايز طاقة في الـGym...',
 'ياكل بطاطس مهروسة.',
 'اللي عايز تتفتح نِفسه على الأكل...',
 'ياكل "شيبسي".',
 'العالم كان محتاج معجزة من زمان،',
 'والمعجزة اتجسدت على صورة إنسان.',
 '!Potatoman\n !Potatoman\n أعزائي المشاهدين،',
 'السلام عليكم ورحمة الله وبركاته،',
 'أهلًا بكم في حلقة جديدة،',
 'من برنامج "الدحّيح"!',
 'عزيزي،',
 'لو ربنا كرمك،',
 'وذهبت إلى "ألمانيا"،',
 'وذهبت إلى حديقة "سانسوسي"\nفي مدينة "بوتسدام"،',
 'وزُرت كنيسة "السلام" هناك،',
 'هتلاقي قبور مجموعة من ملوك "ألمانيا"،',
 'منهم قبر واحد\nمن أهم ملوك "بروسيا" القرن الـ18،',
 'الملك "فريدريك التاني"،',
 'المعروف بـ"

# <p style="padding:50px;background-color:#DA8359;margin:0;color:#fafefe;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:100">EDA</p>

## Visualization

## Questions

# <p style="padding:50px;background-color:#DA8359;margin:0;color:#fafefe;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:100">Cleaning</p>

## Tidying Up Text

### Orthographic mistakes

### Spelling inconsistencies (Text Correction)

In [57]:
autoco = AutoCorrector()
text = 'اذا أردت إستعارة كتاب ، اذهب الى المكتبه او الادارة فى الضهيرة .'
autoco.spell(text)

'إذا أردت استعارة كتاب، اذهب إلى المكتبة أو الادارة في الظهيرة.'

### Unknown characters

### Repeated letters and with spaces in the words



## Text Processing

### Normalization

> <span style="color: green">**_Normalization:_**</span> 

### Tokenization & Segmentation

> <span style="color: green">**_Tokenization:_**</span> is the process of breaking a sequence of text into smaller units called tokens, such as words, phrases, symbols, and other elements. For the Arabic language, tokenization is a complex task due to the differences between the written and spoken forms of the language.

### Stemming

> <span style="color: green">**_Stemming:_**</span> 

### Lemmatization

> <span style="color: green">**_Lemmatization:_**</span> 

### Stop Words Removal

> <span style="color: green">**_Stop Words Removal:_**</span> 

### Punctuation

> <span style="color: green">**_Punctuation:_**</span> 

### Part-of-speech tagging

> <span style="color: green">**_Part-of-speech tagging:_**</span> 

### Named entity recognition (NER)

> <span style="color: green">**_Named entity recognition:_**</span> find and label named entities like proper nouns, organisations, places, etc.

### Corpus and lexical resources

> <span style="color: green">**_Corpus and lexical resources:_**</span> annotated corpora and lexical databases that can be used for tasks like language modelling and information retrieval.

## Handling Outliers

# <p style="padding:50px;background-color:#DA8359;margin:0;color:#fafefe;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:100">Feature Engineering</p>

# <p style="padding:50px;background-color:#DA8359;margin:0;color:#fafefe;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:100">Preporcessing</p>

### Text Classification

### Word Embedding

### Multi-Label Labelling

### Topic Modeling

# <p style="padding:50px;background-color:#DA8359;margin:0;color:#fafefe;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:100">Models</p>

### AraBert

In [30]:
from arabert import ArabertPreprocessor
from arabert.aragpt2.grover.modeling_gpt2 import GPT2LMHeadModel

from arabert.preprocess import ArabertPreprocessor

model_name = "aubmindlab/bert-base-arabertv2"
arabert_prep = ArabertPreprocessor(model_name=model_name)

text = "ولن نبالغ إذا قلنا: إن 'هاتف' أو 'كمبيوتر المكتب' في زمننا هذا ضروري"
arabert_prep.preprocess(text)

c:\Users\mazen\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\mazen\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


100%|██████████| 241M/241M [07:15<00:00, 554kiB/s] 


[2025-02-11 03:11:01,230 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


"? ?? ????? ??? ?? ?? : ?? ' ???? ' ?? ' ??????? ?? ???? ' ?? ??? ?? ??? ?????"

In [32]:
arabert_prep.preprocess("أهلا")

'??? ?'

# <p style="padding:50px;background-color:#DA8359;margin:0;color:#fafefe;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:100">Resources</p>